In [ ]:
# prepare database table

# batch evaluation of all reference midis + store results in db (only similarity distances or also absolute values?)

# calculate the average + store it as reference for normalization
# => might be worth changing architecture to object orientation here (creating an instance of the evaluation class, where you can set certain reference parameters)